In [86]:
import pandas as pd

In [87]:
df = pd.read_parquet('./data/data_df_hcm.parquet')
target_df = pd.read_parquet('./data/target_df_hcm.parquet')


In [88]:
df

,nearest_5_district,nearest_8_street,nearest_2_ward,nearest_4_district,nearest_1_district,typeOfRealEstate,houseDirection,nearest_5_street,nearest_7_street,street,...,distance_hn_CongtruongLamSon,distance_hn_DuongNguyenHuequan1TPHCM,nearest_4_lat,num_of_atm_in_2000m_radius,num_of_place_of_worship_in_2000m_radius,num_of_restaurant_in_1000m_radius,numberOfBathRooms,distance_hn_DuongNamKyKhoiNghia,event_timestamp,realestate_id
24794,11,1730,13,11,11,3,0,857,756,253,...,7.647704,7.771465,10.790462,106,98,97,4.478184,7.835349,2023-04-01,24794
37176,10,2986,547,10,10,3,0,4053,3429,493,...,8.551525,8.572820,10.786360,35,72,15,4.478184,8.578963,2023-04-03,37176
6905,11,100,518,11,11,3,0,495,100,2252,...,7.143411,6.899570,10.766617,129,109,303,4.478184,6.688504,2023-05-18,6905
6412,10,3640,438,14,14,0,1,3062,1479,1551,...,8.428535,8.488024,10.803249,58,48,13,2.000000,8.519420,2023-07-13,6412
24436,18,1998,28,18,18,1,4,208,2642,219,...,8.776880,8.799135,10.799805,7,9,9,3.000000,8.820474,2023-07-14,24436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697,1,357,116,1,1,1,0,3299,506,153,...,8.728899,8.680754,10.736959,9,2,3,2.000000,8.654018,2024-02-16,2697
22001,20,963,30,14,20,3,0,91,311,30,...,8.216211,8.298848,10.805246,57,55,15,5.000000,8.345146,2024-02-16,22001
21481,13,2268,399,13,13,1,7,2372,1469,2733,...,9.479880,9.503459,10.886026,3,1,0,1.000000,9.516926,2024-02-16,21481
10914,10,599,140,30,10,3,0,3708,3451,1047,...,8.967848,8.989615,10.805946,17,12,3,4.478184,8.998477,2024-02-16,10914


In [89]:
target_df

,target,event_timestamp,realestate_id
24794,450.000000,2023-04-01,24794
37176,176.000000,2023-04-03,37176
6905,406.250000,2023-05-18,6905
6412,69.863014,2023-07-13,6412
24436,185.000000,2023-07-14,24436
...,...,...,...
2697,96.250000,2024-02-16,2697
22001,123.595506,2024-02-16,22001
21481,51.875000,2024-02-16,21481
10914,257.352941,2024-02-16,10914


In [90]:
import json

FS = json.load(open('./data/full_version.json', 'r'))
cat_cols = FS['cat_cols']
num_cols = FS['num_cols']
all_cols = cat_cols + num_cols

df = df[all_cols]

In [91]:
categorical_features_indices = [i for i, c in enumerate(all_cols) if c in cat_cols]

In [92]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import random

def create_model(cat_idxs = None):
    if cat_idxs:
        model = CatBoostRegressor(cat_features=cat_idxs, iterations=5000, verbose = 100, random_state=random.randint(3, 1000), task_type='GPU', devices='0:1')
    else:
        model = CatBoostRegressor(iterations=5000, verbose = 100, random_state=random.randint(3, 1000), task_type='GPU', devices='0:1')
    search_params = {'learning_rate': [0.01, 0.08, 0.15]}
    clf = GridSearchCV(model, search_params, scoring=['explained_variance', 'max_error', 'neg_root_mean_squared_error', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_percentage_error'], refit='neg_root_mean_squared_error', cv=5)
    return clf

In [93]:
model = create_model(categorical_features_indices)

In [94]:
full_df = pd.concat([df, target_df['target']], axis = 1)

In [95]:
full_df.shape

(44209, 184)

In [96]:
train_df = full_df.iloc[:-35000]
test_df = full_df.iloc[-35000:]

In [97]:
def train_test_split_by_col(train_df, test_df, X_cols, y_col):
    X_train, X_test, y_train, y_test = train_df[X_cols], test_df[X_cols], train_df[y_col], test_df[y_col]

    return X_train, X_test, y_train, y_test


In [98]:
target_feature = 'target'

In [99]:
X_train, X_test, y_train, y_test = train_test_split_by_col(train_df = train_df, test_df = test_df, X_cols = all_cols, y_col = target_feature)
model.fit(X_train, y_train)

0:	learn: 129.5994812	total: 121ms	remaining: 10m 7s
100:	learn: 83.4924293	total: 7.25s	remaining: 5m 51s
200:	learn: 71.6738386	total: 13.7s	remaining: 5m 26s
300:	learn: 67.8822870	total: 19.6s	remaining: 5m 5s
400:	learn: 66.1782580	total: 25.8s	remaining: 4m 55s
500:	learn: 65.0346724	total: 31.8s	remaining: 4m 45s
600:	learn: 64.0554133	total: 38s	remaining: 4m 37s
700:	learn: 63.2615665	total: 41s	remaining: 4m 11s
800:	learn: 62.6292393	total: 44.1s	remaining: 3m 50s
900:	learn: 62.1062813	total: 47.1s	remaining: 3m 34s
1000:	learn: 61.4217728	total: 50.2s	remaining: 3m 20s
1100:	learn: 60.8747175	total: 53.3s	remaining: 3m 8s
1200:	learn: 60.2760042	total: 56.4s	remaining: 2m 58s
1300:	learn: 59.8345146	total: 59.5s	remaining: 2m 49s
1400:	learn: 59.4237817	total: 1m 2s	remaining: 2m 40s
1500:	learn: 59.0212074	total: 1m 5s	remaining: 2m 32s
1600:	learn: 58.6108822	total: 1m 8s	remaining: 2m 25s
1700:	learn: 58.2361570	total: 1m 11s	remaining: 2m 18s
1800:	learn: 57.9201695	to

0:	learn: 127.6950351	total: 42.8ms	remaining: 3m 33s
100:	learn: 82.1982989	total: 5.5s	remaining: 4m 26s
200:	learn: 70.4922097	total: 12s	remaining: 4m 45s
300:	learn: 66.7529761	total: 19.4s	remaining: 5m 2s
400:	learn: 64.9865843	total: 26s	remaining: 4m 58s
500:	learn: 63.8466119	total: 32.2s	remaining: 4m 48s
600:	learn: 62.8654550	total: 38.2s	remaining: 4m 39s
700:	learn: 62.0587040	total: 44.2s	remaining: 4m 30s
800:	learn: 61.3364705	total: 49.8s	remaining: 4m 21s
900:	learn: 60.6927051	total: 56.2s	remaining: 4m 15s
1000:	learn: 60.0776227	total: 1m 2s	remaining: 4m 10s
1100:	learn: 59.5486530	total: 1m 8s	remaining: 4m 4s
1200:	learn: 59.0807942	total: 1m 15s	remaining: 3m 58s
1300:	learn: 58.6296153	total: 1m 21s	remaining: 3m 51s
1400:	learn: 58.2313575	total: 1m 28s	remaining: 3m 46s
1500:	learn: 57.8613462	total: 1m 34s	remaining: 3m 40s
1600:	learn: 57.4992331	total: 1m 40s	remaining: 3m 34s
1700:	learn: 57.2176586	total: 1m 46s	remaining: 3m 26s
1800:	learn: 56.91084

0:	learn: 129.4832783	total: 150ms	remaining: 12m 28s
100:	learn: 82.5661821	total: 14.5s	remaining: 11m 44s
200:	learn: 70.4414940	total: 28.9s	remaining: 11m 29s
300:	learn: 66.5739775	total: 42.8s	remaining: 11m 7s
400:	learn: 64.7752921	total: 56.7s	remaining: 10m 50s
500:	learn: 63.6133013	total: 1m 11s	remaining: 10m 46s
600:	learn: 62.6042641	total: 1m 26s	remaining: 10m 31s
700:	learn: 61.8226335	total: 1m 40s	remaining: 10m 19s
800:	learn: 61.1281701	total: 1m 55s	remaining: 10m 6s
900:	learn: 60.5919983	total: 2m 10s	remaining: 9m 53s
1000:	learn: 59.9897213	total: 2m 28s	remaining: 9m 53s
1100:	learn: 59.3919692	total: 2m 42s	remaining: 9m 36s
1200:	learn: 58.8888239	total: 2m 57s	remaining: 9m 21s
1300:	learn: 58.4766436	total: 3m 11s	remaining: 9m 3s
1400:	learn: 58.1253963	total: 3m 25s	remaining: 8m 48s
1500:	learn: 57.7199913	total: 3m 39s	remaining: 8m 31s
1600:	learn: 57.3613614	total: 3m 53s	remaining: 8m 16s
1700:	learn: 57.0314890	total: 4m 11s	remaining: 8m 7s
180

0:	learn: 126.2861757	total: 49.6ms	remaining: 4m 8s
100:	learn: 80.8072345	total: 6.67s	remaining: 5m 23s
200:	learn: 69.2242379	total: 15.2s	remaining: 6m 3s
300:	learn: 65.5422920	total: 21.5s	remaining: 5m 34s
400:	learn: 63.7939600	total: 28.4s	remaining: 5m 25s
500:	learn: 62.7415252	total: 35.1s	remaining: 5m 15s
600:	learn: 61.8849188	total: 42s	remaining: 5m 7s
700:	learn: 61.1138146	total: 48.3s	remaining: 4m 55s
800:	learn: 60.4217430	total: 55.3s	remaining: 4m 49s
900:	learn: 59.7949462	total: 1m 1s	remaining: 4m 40s
1000:	learn: 59.1321518	total: 1m 8s	remaining: 4m 34s
1100:	learn: 58.6236255	total: 1m 15s	remaining: 4m 27s
1200:	learn: 58.1667282	total: 1m 21s	remaining: 4m 19s
1300:	learn: 57.7803703	total: 1m 28s	remaining: 4m 11s
1400:	learn: 57.3825061	total: 1m 34s	remaining: 4m 4s
1500:	learn: 57.0143855	total: 1m 41s	remaining: 3m 57s
1600:	learn: 56.6095436	total: 1m 48s	remaining: 3m 50s
1700:	learn: 56.2674319	total: 1m 55s	remaining: 3m 43s
1800:	learn: 56.000

0:	learn: 129.6457798	total: 63.6ms	remaining: 5m 17s
100:	learn: 82.2344936	total: 6.82s	remaining: 5m 30s
200:	learn: 70.3354650	total: 13.2s	remaining: 5m 14s
300:	learn: 66.7796870	total: 19.6s	remaining: 5m 6s
400:	learn: 65.0808025	total: 26.1s	remaining: 4m 59s
500:	learn: 63.9547907	total: 32.8s	remaining: 4m 54s
600:	learn: 62.9918906	total: 39.6s	remaining: 4m 50s
700:	learn: 62.2638775	total: 46.5s	remaining: 4m 44s
800:	learn: 61.4967933	total: 52.8s	remaining: 4m 36s
900:	learn: 60.8530987	total: 59.1s	remaining: 4m 28s
1000:	learn: 60.1697450	total: 1m 6s	remaining: 4m 25s
1100:	learn: 59.5571218	total: 1m 13s	remaining: 4m 19s
1200:	learn: 58.9512211	total: 1m 28s	remaining: 4m 39s
1300:	learn: 58.4050683	total: 1m 34s	remaining: 4m 29s
1400:	learn: 57.9195101	total: 1m 41s	remaining: 4m 20s
1500:	learn: 57.4869220	total: 1m 47s	remaining: 4m 11s
1600:	learn: 57.1180678	total: 1m 56s	remaining: 4m 7s
1700:	learn: 56.7341745	total: 2m 12s	remaining: 4m 17s
1800:	learn: 56

...

0:	learn: 117.6343538	total: 76.9ms	remaining: 6m 24s
100:	learn: 58.1965701	total: 6.46s	remaining: 5m 13s
200:	learn: 53.8387232	total: 13.1s	remaining: 5m 12s
300:	learn: 51.6462553	total: 18.6s	remaining: 4m 50s
400:	learn: 50.1414857	total: 24.9s	remaining: 4m 45s
500:	learn: 49.4124457	total: 31s	remaining: 4m 38s
600:	learn: 48.6319760	total: 37.1s	remaining: 4m 31s
700:	learn: 47.9083669	total: 42.8s	remaining: 4m 22s
800:	learn: 47.2720366	total: 49.4s	remaining: 4m 18s
900:	learn: 46.7681480	total: 55.6s	remaining: 4m 13s
1000:	learn: 46.2701169	total: 1m 2s	remaining: 4m 7s
1100:	learn: 45.6026818	total: 1m 8s	remaining: 4m 3s
1200:	learn: 44.9365675	total: 1m 16s	remaining: 4m
1300:	learn: 44.2905112	total: 1m 22s	remaining: 3m 55s
1400:	learn: 43.9381051	total: 1m 28s	remaining: 3m 48s
1500:	learn: 43.4248648	total: 1m 35s	remaining: 3m 42s
1600:	learn: 42.9528817	total: 1m 42s	remaining: 3m 36s
1700:	learn: 42.6836533	total: 1m 47s	remaining: 3m 29s
1800:	learn: 42.491630

0:	learn: 114.8174252	total: 34.3ms	remaining: 2m 51s
100:	learn: 57.3910427	total: 6.07s	remaining: 4m 54s
200:	learn: 54.2811108	total: 12s	remaining: 4m 45s
300:	learn: 52.5152528	total: 18.7s	remaining: 4m 51s
400:	learn: 50.9313823	total: 24.5s	remaining: 4m 40s
500:	learn: 49.7990342	total: 31s	remaining: 4m 37s
600:	learn: 49.1007502	total: 37.1s	remaining: 4m 31s
700:	learn: 48.1018444	total: 43.5s	remaining: 4m 27s
800:	learn: 47.4623696	total: 50s	remaining: 4m 22s
900:	learn: 46.8253428	total: 57s	remaining: 4m 19s
1000:	learn: 46.4905676	total: 1m 2s	remaining: 4m 11s
1100:	learn: 45.9047547	total: 1m 9s	remaining: 4m 5s
1200:	learn: 45.4219931	total: 1m 15s	remaining: 3m 57s
1300:	learn: 45.0686459	total: 1m 21s	remaining: 3m 51s
1400:	learn: 44.8623477	total: 1m 27s	remaining: 3m 45s
1500:	learn: 44.6078459	total: 1m 34s	remaining: 3m 39s
1600:	learn: 44.2251756	total: 1m 40s	remaining: 3m 34s
1700:	learn: 43.9077231	total: 1m 48s	remaining: 3m 29s
1800:	learn: 43.6755067

0:	learn: 117.6812069	total: 58.6ms	remaining: 4m 53s
100:	learn: 58.0063879	total: 6.29s	remaining: 5m 4s
200:	learn: 53.7735126	total: 13s	remaining: 5m 9s
300:	learn: 51.4210388	total: 19.8s	remaining: 5m 9s
400:	learn: 49.9589028	total: 26.2s	remaining: 5m
500:	learn: 48.6396151	total: 32.8s	remaining: 4m 54s
600:	learn: 47.5986361	total: 39.9s	remaining: 4m 51s
700:	learn: 46.8130088	total: 46.4s	remaining: 4m 44s
800:	learn: 46.3866331	total: 52.8s	remaining: 4m 36s
900:	learn: 45.8042855	total: 59.3s	remaining: 4m 29s
1000:	learn: 45.3861239	total: 1m 6s	remaining: 4m 24s
1100:	learn: 44.9810040	total: 1m 12s	remaining: 4m 15s
1200:	learn: 44.5597234	total: 1m 18s	remaining: 4m 8s
1300:	learn: 44.1300116	total: 1m 24s	remaining: 4m
1400:	learn: 43.8476845	total: 1m 31s	remaining: 3m 53s
1500:	learn: 43.3395897	total: 1m 37s	remaining: 3m 47s
1600:	learn: 42.9731092	total: 1m 43s	remaining: 3m 40s
1700:	learn: 42.6298330	total: 1m 50s	remaining: 3m 33s
1800:	learn: 42.4118121	tot

0:	learn: 122.6019783	total: 43.5ms	remaining: 3m 37s
100:	learn: 61.4578597	total: 6.18s	remaining: 4m 59s
200:	learn: 57.4336766	total: 12.3s	remaining: 4m 54s
300:	learn: 54.8489847	total: 18.7s	remaining: 4m 51s
400:	learn: 53.0777862	total: 25.8s	remaining: 4m 55s
500:	learn: 51.9747385	total: 32.4s	remaining: 4m 50s
600:	learn: 51.0743264	total: 38.5s	remaining: 4m 41s
700:	learn: 50.4289042	total: 45.2s	remaining: 4m 37s
800:	learn: 49.9582582	total: 52.4s	remaining: 4m 34s
900:	learn: 49.4246405	total: 59.3s	remaining: 4m 29s
1000:	learn: 49.0418389	total: 1m 5s	remaining: 4m 22s
1100:	learn: 48.6898856	total: 1m 11s	remaining: 4m 14s
1200:	learn: 48.3211422	total: 1m 17s	remaining: 4m 6s
1300:	learn: 47.9047994	total: 1m 24s	remaining: 4m
1400:	learn: 47.5657366	total: 1m 30s	remaining: 3m 53s
1500:	learn: 47.0451873	total: 1m 37s	remaining: 3m 46s
1600:	learn: 46.7270535	total: 1m 43s	remaining: 3m 40s
1700:	learn: 46.3260410	total: 1m 50s	remaining: 3m 34s
1800:	learn: 46.04

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f8180035e70>,
             param_grid={'learning_rate': [0.01, 0.08, 0.15]},
             refit='neg_root_mean_squared_error',
             scoring=['explained_variance', 'max_error',
                      'neg_root_mean_squared_error', 'r2',
                      'neg_median_absolute_error',
                      'neg_mean_absolute_percentage_error'])

In [100]:
model.predict(X_test)[:10]

array([ 72.84532565, 165.10174401, 126.172117  ,  62.51216053,
        39.47311841,  85.87211974,  33.29281679, 245.01268579,
       166.78044912, 106.04345187])

In [101]:
from joblib import dump, load

In [102]:
estimator = model.best_estimator_
dump(estimator, "./model/hcm/cat/v1/model.joblib")

['./model/hcm/cat/v1/model.joblib']

In [103]:
load_model = load("./model/hcm/cat/v1/model.joblib")
load_model.predict(X_test)[:10]

array([ 72.84532565, 165.10174401, 126.172117  ,  62.51216053,
        39.47311841,  85.87211974,  33.29281679, 245.01268579,
       166.78044912, 106.04345187])